<a href="https://colab.research.google.com/github/mandemadhura/crewai_exercises/blob/main/aiagent_elements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [44]:
import os
# set OPENAI KEY
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

In [88]:
reader = Agent(
    role="Reader",
	  goal="Extract accurate and to the point "
        "information from a {file}",
	  backstory=(
		"You are a very good reader and expert at retrieving key points for a related {question} "
		"Your job is to provide support to a {customer}, a super important customer "
        " for the queries related to a content from a {file}"
		"Make sure to provide full complete answers, bullet points if that is available "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [89]:
support_quality_assurance_agent = Agent(
	role="Quality Assurance Specialist",
	goal="Make sure that customer question is answered with accuracy",
	backstory=(
		"on a request from {customer} ensuring that the reader is "
		"providing the best support possible with accurate answers from a {file}.\n"
		"You need to make sure that the reader is providing full"
		"complete answers, and make no assumptions. Also make sure that the provided "
		"Strictly, answer must be from the {file} only and should not have any "
		"other info from any other source."
	),
	verbose=True
)

In [90]:
from crewai_tools import tool

@tool("file_reader")
def file_reader(file: str):
  """Opens and read a file and return the content"""
  with open(file=file, mode="rb") as f:
    return f.read()
#pdf_reader = FileReadTool(path="common-yoga-protocol.pdf")

In [92]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
            "{question}\n\n"
        "{person} from {customer} is the one that reached out. "
                "Make sure to use everything you know "
        "to provide the best support possible."
                "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly "
        "tone throughout. Give in bullets if available."
    ),
    tools=[file_reader],
    agent=reader,
)

In [93]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Reader for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and strictly the"
        "answer given is only from the contents in {file} and it should "
        "not contain anything other than mentioned in the {file}.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		    "thoroughly, with a helpful and friendly tone.\n"
		    "ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry and should contain answer from {file} only. "
        "Make sure that answer must not have any other info than from a {file} "
        "regarding the question asked. review this strictly"
	      "Maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [94]:
crew = Crew(
  agents=[reader, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [95]:
inputs = {
    "customer": "Yoga Learner",
    "person": "Madhura",
    "question": "What is Tadasana ",
    "file": "yoga_info.txt"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Reader
 [INFO]: == Starting Task: Yoga Learner just reached out with a super important ask:
What is Tadasana 

Madhura from Yoga Learner is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to retrieve the information about Tadasana from the yoga_info.txt file to provide a detailed and informative response to Madhura's inquiry.

Action: file_reader
Action Input: {"file": "yoga_info.txt"} 

b'Tadasana: T\xc4\x81da means palm tree or mountain. This asana\r\nteaches one to attain stability and firmness and\r\nforms the base for all the standing asana.\r\nTechnique\r\no Stand with feet 2 inches apart.\r\no Interlock the fingers, and turn the wrist\r\noutwards. Now inhale, raise the arms up and\r\nbring them in line with the shoulders.\r\no Raise the heels off the floor and bala

In [96]:
from IPython.display import Markdown
Markdown(result)

Dear Madhura,

Tadasana, also known as Mountain Pose, is a fundamental yoga pose that focuses on stability and firmness. The name translates to palm tree or mountain, symbolizing strength and grounding. It serves as the foundation for standing asanas, emphasizing stability and firmness in the practice of yoga.

Technique:
- Stand with feet 2 inches apart.
- Interlock the fingers and turn the wrist outwards.
- Inhale, raise the arms up and bring them in line with the shoulders.
- Raise the heels off the floor and balance on the toes for 10-15 seconds.
- Exhale, bring the heels down.
- Release the interlock of the fingers and bring the arms down parallel to the trunk, coming back to the standing posture.

Tadasana is a key pose in yoga practice that helps improve posture, balance, and focus. Its name reflects its essence, embodying the strength and stability of a mountain. Practice Tadasana regularly to experience its full benefits in your yoga journey.

Warm regards,
[Your Name]